In [1]:
from __future__ import division, print_function, absolute_import

import numpy as np
import tensorflow as tf
import logging
from DataPrep.data_io import DataFormatter
from nn.load_params import layer_name, convShape, getWeights
from nn.utils import getTriplets, tripletLoss
# from nn.model import initNetwork

logging.basicConfig(level=logging.DEBUG, filename="logfile.log", filemode="w",
                    format="%(asctime)-15s %(levelname)-8s %(message)s")

In [2]:
def reset_graph():  # Reset the graph
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

## GET INCEPTION WEIGHTS

In [3]:
parentPath = "/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Models/FaceNet-Inception"
moduleWeightDict = getWeights(parentPath)

## INITIALIZE NETWORK WITHWEIGHTS

In [4]:
# tensorDict = initNetwork(moduleWeightDict)

## LOAD THE BATCH DATA:

In [29]:
from itertools import combinations

def getTriplets(batch_embedding):
    img_per_label=6
    num_labels=3 
    alpha=0.01
    batch_tripet_idx = []
    idx_arr = np.arange(len(batch_embedding))
    for i in np.arange(num_labels):
        pos_idxs = np.arange( i *img_per_label , i * img_per_label +img_per_label)
        neg_idxs = np.setdiff1d(idx_arr, pos_idxs)
        # print(pos_idxs)
        # print('')
        # print(neg_idxs)
        compare_point = -1  # used to avoid redundancy in calculating SSE between anchor and all negative
        # Get all combination of Anchor and positive
        # print ('######################################')
        for anc_idx, pos_idx in combinations(pos_idxs, 2):
            if anc_idx != compare_point:
                compare_point += 1
                # Get the sum of squared distance between anchor and positive
                anc_VS_neg_ndarr = np.sum(np.square(
                        batch_embedding[anc_idx] - batch_embedding[neg_idxs]), 1
                )
                # print (anc_VS_neg_ndarr)
            
            # Get the sum of squared distance between anchor and positive
            anc_VS_pos = np.sum(
                    np.square(batch_embedding[anc_idx] - batch_embedding[pos_idx]))
            # print ('anc_VS_pos anc_VS_pos', anc_VS_pos)
            
            hard_neg_idx = np.where(anc_VS_neg_ndarr - anc_VS_pos < alpha)[0]
            # print (hard_neg_idx)
            
            # Randomly sample 1 record from the hard negative idx, and create a triplet
            if len(hard_neg_idx) >0:
                # print('hard_neg_idx hard_neg_idx ', hard_neg_idx)
                np.random.shuffle(hard_neg_idx)
                rnd_idx = hard_neg_idx[0]
                # print ('neg_idx neg_idx ', neg_idxs[rnd_idx])
                
                # Get triplet indexes in order to work on offline mode
                batch_tripet_idx.append([anc_idx, pos_idx, neg_idxs[rnd_idx]])
                # print (anc_idx,pos_idx)
                # print ('')
                # print (break_point - cnt)
    # print('TRIPLET INDEX ', batch_tripet_idx)
    
    return [batch_tripet_idx]


def loss(encodingDict):
    tripletIDX = tf.py_func(getTriplets, [encodingDict['output']], tf.int64)
    loss = tripletLoss(
        tf.gather(tf.cast(encodingDict['output'], dtype=tf.float32), tripletIDX[:,0]),
        tf.gather(tf.cast(encodingDict['output'], dtype=tf.float32), tripletIDX[:,1]),
        tf.gather(tf.cast(encodingDict['output'], dtype=tf.float32), tripletIDX[:,2]),
        alpha=0.2)
    encodingDict['tripletIDX'] = tripletIDX
    encodingDict['loss'] = loss
    return encodingDict

In [30]:
# def getBatchTriplets():
#     getTriplets_TF(batch_embedding, img_per_label, num_labels, alpha=0.01)
    
from nn.model import getModel_FT, getModel
def initNetwork(weightDict, isTrainable=False):
    logging.info('INITIALIZING THE NETWORK !! ...............................')
    if not isTrainable:
        encodingDict = getModel([96, 96, 3], params=weightDict)
    else:
        encodingDict = getModel_FT([96, 96, 3], params=weightDict)   
        encodingDict = loss(encodingDict)
    return encodingDict

In [31]:
path_to_batch_data = '/Users/sam/All-Program/App-DataSet/DeepFaceRecognition/data_models/batch_img_arr'
batch_file_name = 'random_batches.pickle'

dataX, dataY, labelDict = DataFormatter.getPickleFile(folderPath=path_to_batch_data, 
                                                      picklefileName=batch_file_name, getStats=True)


def optimize(loss):
    opt = tf.train.AdamOptimizer(learning_rate=0.01, beta1=0.9, beta2=0.999, epsilon=0.1)
    grads = opt.compute_gradients(loss, tf.global_variables())
    apply_gradient_op = opt.apply_gradients(grads)
    return apply_gradient_op

reset_graph()
encodingDict = initNetwork(moduleWeightDict, isTrainable=True)
print (encodingDict)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for batchNum, batchX in enumerate(dataX):
        batchY = dataY[batchNum, :]
        embeddings_idx, anchor = sess.run([encodingDict['tripletIDX'], 
                                   encodingDict['loss']], 
                                  feed_dict={encodingDict['inpTensor']:batchX})
        print (embeddings_idx)
        print('@@@@')
        print (anchor)
        print ('#########################')
#         embeddings = embeddings[0]
        
#         a
#         tripletIndexArr = np.array(tripletIndexArr).reshape(-1,3)
#         a_idxs = tripletIndexArr[:,0].flatten()
#         p_idxs = tripletIndexArr[:,1].flatten()
#         n_idxs = tripletIndexArr[:,2].flatten()
#         loss = tripletLoss(tf.gather(tf.cast(embeddings, dtype=tf.float32), a_idxs),
#                            tf.gather(tf.cast(embeddings, dtype=tf.float32), p_idxs),
#                            tf.gather(tf.cast(embeddings, dtype=tf.float32), n_idxs), alpha=0.2)
#         print (loss.eval())
        
        
#         pritn ()

The shape of input data (X) is:  (10, 18, 96, 96, 3)
The shape of input data (Y) is:  (10, 18)
Unique labels in dataY is:  [ 0.  1.  2.]
Label dict:  None
{'output': <tf.Tensor 'InceptionFC_FT/L2_norm:0' shape=(?, 128) dtype=float32>, 'loss': <tf.Tensor 'TripletLoss/Sum_2:0' shape=<unknown> dtype=float32>, 'tripletIDX': <tf.Tensor 'PyFunc:0' shape=<unknown> dtype=int64>, 'inpTensor': <tf.Tensor 'Placeholder:0' shape=(?, 96, 96, 3) dtype=float32>}


KeyError: 'anchor'

In [ ]:
import tensorflow as tf
import numpy as np
np.random.seed(1)
tfdata = tf.cast(np.random.rand(1,1,3,5) + 10, dtype=tf.float32)
print (tfdata.get_shape().as_list())
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batchMean, batchVar = tf.nn.moments(tfdata, axes=[0,1,2], name="moments")
    print (tfdata.eval())
    print (batchMean.eval())
    print (batchMean.get_shape().as_list())
    print (batchVar.eval())
    print (batchVar.get_shape().as_list())

In [44]:
np.array([1,2,3,4])[-1]

4

In [12]:
t = tf.constant([[[1, 1, 1], [2, 2, 2]],
                 [[3, 3, 3], [4, 4, 4]],
                 [[5, 5, 5], [6, 6, 6]]])

a = tf.constant([[1,2,3],[3,2,4],[1,1,4]])
print(t.shape)
print (a.shape)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print (tf.gather(a, [0, 1]).eval())



(3, 2, 3)
(3, 3)
[[1 2 3]
 [3 2 4]]


In [13]:
pow(0.3426 - 0.927, 2) + pow(0.7484-0.62853, 2)

0.3558921769